# Pipeline

Data ingestion -> Document Store (Azure AI Search)

In [29]:
import dotenv

# Load environment variables from .env file
dotenv.load_dotenv()

True

## 1. Ingest pdf(s)

Ingest pdf(s) in `/data` folder

In [3]:
import base64

def encode_pdf_to_base64(file_path):
    """
    Reads a PDF file and converts it to a base64 data URI.
    Required because Azure MaaS endpoints usually don't accept local paths.
    """
    with open(file_path, "rb") as pdf_file:
        encoded_string = base64.b64encode(pdf_file.read()).decode("utf-8")
    
    # Mistral expects this exact format prefix
    return f"data:application/pdf;base64,{encoded_string}"

## 2. Run OCR

Run OCR to extract text from each page. Mistral document model (https://docs.mistral.ai/capabilities/document_ai), it is on Azure AI foundry

In [ ]:
import glob
import os
import requests
import io
import base64
from pypdf import PdfReader, PdfWriter

results = []
pdf_files = glob.glob(os.path.join("data", "*.pdf"))

print(f"Found {len(pdf_files)} PDFs. Starting OCR job...\n")

headers = {
    "Authorization": f"Bearer {os.getenv('AZURE_OPENAI_API_KEY')}",
    "Content-Type": "application/json"
}

# Pages per batch request (to avoid azure timeout for large PDFs)
BATCH_SIZE = 5 

for file_path in pdf_files:
    file_name = os.path.basename(file_path)
    print(f"Processing: {file_name}...", end=" ")
    
    try:
        # Split pdf into chunks/batches
        reader = PdfReader(file_path)
        total_pages = len(reader.pages)
        file_page_data = [] # Store all pages for this file here

        # Iterate in batches (e.g., 0-5, 5-10, etc.)
        for start_idx in range(0, total_pages, BATCH_SIZE):
            end_idx = min(start_idx + BATCH_SIZE, total_pages)
            
            # Create a temporary PDF in memory for this batch
            writer = PdfWriter()
            for i in range(start_idx, end_idx):
                writer.add_page(reader.pages[i])
            
            with io.BytesIO() as bytes_stream:
                writer.write(bytes_stream)
                bytes_stream.seek(0)
                encoded_batch = base64.b64encode(bytes_stream.read()).decode("utf-8")
                base64_string = f"data:application/pdf;base64,{encoded_batch}"

            # 1. Prepare Payload (using the batch instead of full file)
            payload = {
                "model": "mistral-document-ai-2505",
                "document": {
                    "type": "document_url",
                    "document_url": base64_string
                },
                "include_image_base64": False 
            }
            
            # 2. Send Request
            response = requests.post(os.getenv("AZURE_MISTRAL_ENDPOINT"), headers=headers, json=payload)
            
            # 3. Handle Response
            if response.status_code == 200:
                data = response.json()
                
                # Combine this batch's pages into the main list
                for i, page in enumerate(data.get('pages', [])):
                    file_page_data.append({
                        "page_num": start_idx + i + 1,  # Calculate correct page number
                        "text": page['markdown']
                    })
            else:
                print(f"\nError on batch {start_idx}-{end_idx}: {response.status_code} - {response.text}")
                break # Stop processing this file if a batch fails
        
        # Only append if we got data
        if file_page_data:
            results.append({
                "source_context": file_name,
                "file_path": file_path,
                "pages": file_page_data 
            })
            print("Done.")
            
    except Exception as e:
        print(f"Failed: {str(e)}")

print("\nAll files processed.")

Found 2 PDFs. Starting OCR job...

Processing: embedding_retrieval.pdf... Done.
Processing: refrag_research.pdf... Done.

All files processed.


## 3. Chunking

Chunk OCR test with a simple simple textsplitter (https://learn.microsoft.com/en-us/azure/search/vector-search-how-to-chunk-documents#langchain-data-chunking-example)

In [13]:
# !pip install langchain-text-splitters
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Configure the splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,   # Characters per chunk (adjust based on your embedding model limit)
    chunk_overlap=500, # overlap keeps context between cuts
    separators=["\n\n", "\n", " ", ""] # Try to split by paragraphs first, then lines, then words
)

In [14]:
chunked_data = []

# iterating over the 'results' list from the previous OCR step
for doc in results:
    filename = doc['source_context']
    
    # Iterate through each PAGE first
    for page in doc['pages']:
        page_num = page['page_num']
        page_text = page['text']
        
        # Split ONLY this page's text
        chunks = splitter.split_text(page_text)
        
        for i, text_chunk in enumerate(chunks):
            chunked_data.append({
                "chunk_id": f"{filename}_p{page_num}_{i}",
                "source": filename,
                "page": page_num,
                "text": text_chunk
            })

print(f"Generated {len(chunked_data)} chunks with page numbers.")

Generated 116 chunks with page numbers.


In [17]:
# Preview the first 2 chunks
for chunk in chunked_data[:2]:
    print(f"Chunk from {chunk['source']}")
    print(chunk['text'][:150] + "...") # Print first 150 chars
    print("\n")

Chunk from embedding_retrieval.pdf
On the Theoretical Limitations of Embedding-Based Retrieval

Orion Weller^{*,1,2}, Michael Boratko^{1}, Iftekhar Naim^{1} and Jinhyuk Lee^{1}

^{1}Goo...


Chunk from embedding_retrieval.pdf
In recent years this has been pushed even further with the rise of instruction-following retrieval benchmarks, where models are asked to represent any...




## 4. Embedding

Generate vector embeddings per chunk using the Azure OpenAI embedding model. (https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/embeddings?view=foundry-classic&tabs=csharp)

In [18]:
from openai import AzureOpenAI
import os

# Setup Client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

def get_embedding(text):
    text = text.replace("\n", " ") # Clean newlines to avoid token weirdness
    return client.embeddings.create(
        input=[text], 
        model="text-embedding-3-small",
    ).data[0].embedding

# Apply to all chunks
print(f"Embedding {len(chunked_data)} chunks...")

for i, chunk in enumerate(chunked_data):
    try:
        vector = get_embedding(chunk['text'])
        chunk['values'] = vector # Store the 3072 float list
        
        if i % 10 == 0: print(f".", end="") # Progress bar
        
    except Exception as e:
        print(f"\nError on chunk {i}: {e}")

print("\nDone! Embeddings generated.")

Embedding 116 chunks...
............
Done! Embeddings generated.


## 5. Vector DB

Index in Azure AI Search: store chunk text + metadata (document id, page number, folder, category, source_link) + embedding vector; enable vector search.

get data ready for upload

In [19]:
import uuid

documents_to_upload = []

print(f"Preparing payload from {len(chunked_data)} chunks...")

for chunk in chunked_data:
    # Map to your Azure Search Index Schema
    azure_doc = {
        "id": str(uuid.uuid4()),
        "content": chunk['text'],
        "contentVector": chunk['values'],
        # Citation will look like: "Source: report.pdf (Page 4)"
        "location": f"Source: {chunk['source']} (Page {chunk['page']})" 
    }
    
    documents_to_upload.append(azure_doc)

print(f"Ready to upload {len(documents_to_upload)} documents.")

Preparing payload from 116 chunks...
Ready to upload 116 documents.


upload to AI search

In [28]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

# Initialize Client
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_PRIMARY_API_KEY"))
client = SearchClient(endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
                      index_name=os.getenv("AZURE_SEARCH_INDEX_NAME"),
                      credential=credential)

# Upload in batches (Azure has a limit of ~1000 docs per request)
BATCH_SIZE = 1000
for i in range(0, len(documents_to_upload), BATCH_SIZE):
    batch = documents_to_upload[i : i + BATCH_SIZE]
    
    try:
        result = client.upload_documents(documents=batch)
        print(f"Uploaded batch {i} - {i+len(batch)}: Success")
    except Exception as e:
        print(f"Error uploading batch {i}: {e}")

print("Upload Complete.")

Uploaded batch 0 - 116: Success
Upload Complete.


## 6. Testing

Validate end-to-end: run a few test queries, confirm top results point back to the right page/chunk, and iterate on chunking/cleaning. 

without mcp

In [5]:
import os
from openai import AzureOpenAI, OpenAI
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
import sys

# Embedding Client (Azure OpenAI - for converting query to vector)
embedding_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# 2. Search Client (Azure AI Search - for finding relevant docs)
search_client = SearchClient(
    endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
    index_name=os.getenv("AZURE_SEARCH_INDEX_NAME"),
    credential=AzureKeyCredential(os.getenv("AZURE_SEARCH_API_KEY")) # this is using the query key, use primary key if it doesnt work
)

# 3. Chat Client (Mistral on Azure MaaS)
chat_client = OpenAI(
    base_url=os.getenv("AZURE_OPENAI_INFERENCE"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

def retrieve_context(query_text):
    print("Generating query embedding...", end=" ")
    # Generate Vector for the user's query
    embedding_response = embedding_client.embeddings.create(
        input=query_text,
        model="text-embedding-3-small"
    )
    query_vector = embedding_response.data[0].embedding
    print("Done.")

    print("Searching Vector Index...", end=" ")
    # Perform Vector Search
    vector_query = VectorizedQuery(
        vector=query_vector, 
        k_nearest_neighbors=3, 
        fields="contentVector"
    )
    
    results = search_client.search(
        search_text=query_text, # Hybrid search (keywords + vector)
        vector_queries=[vector_query],
        select=["content", "location"],
        top=3
    )

    # Format results as a single string
    context_parts = []
    for result in results:
        context_parts.append(f"Source: {result['location']}\nContent: {result['content']}")
    
    print(f"Found {len(context_parts)} relevant chunks.")
    return "\n\n".join(context_parts)


# Main
user_input = input("\nWhat would you like to know? (leave empty if you want to select from predefined test queries)")

# use predefined test query
if user_input == "":
    no_input_question = input("[1] What is the primary trade-off in RAG systems that REFRAG aims to solve? [2] What is the Time-To-First-Token (TTFT) acceleration achieved by REFRAG compared to LLaMA? [3] How does REFRAG use curriculum learning for the reconstruction task?")
    
    if no_input_question == "1":
        user_input = "What is the primary trade-off in RAG systems that REFRAG aims to solve?"
    elif no_input_question == "2":
        user_input = "What is the Time-To-First-Token (TTFT) acceleration achieved by REFRAG compared to LLaMA?"
    elif no_input_question == "3":
        user_input = "How does REFRAG use curriculum learning for the reconstruction task?"
    else:
        print("no query given")
        sys.exit()

# Get relevant context
retrieved_context = retrieve_context(user_input)

# Define System Prompt
system_prompt = """You are a helpful assistant. Use the provided 'Context' to answer the user's question.
If the answer is not in the context, say you don't know.
Always cite your sources using the format [Source: filename, Page: page]."""

# Call Mistral with Context
completion = chat_client.chat.completions.create(
    model="Mistral-Large-3",
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "user", 
            "content": f"Context:\n{retrieved_context}\n\nQuestion: {user_input}"
        }
    ],
)

print("\nAnswer:")
print(completion.choices[0].message.content)

Generating query embedding... Done.
Searching Vector Index... Found 3 relevant chunks.

Answer:
REFRAG employs curriculum learning for the reconstruction task by incrementally increasing the difficulty of the task to help the model gradually acquire complex skills. Specifically:

1. **Starting Simple**: The training begins with reconstructing a single chunk. The encoder receives one chunk embedding \(\mathbf{c}_{1}\) for \(x_{1:k}\), and the decoder reconstructs the \(k\) tokens using the projected chunk embedding \(\mathbf{e}_{1}^{\text{cnk}}\).

2. **Gradual Complexity**: The model then progresses to reconstructing longer sequences, such as \(x_{1:2k}\) from \(\mathbf{e}_{1}^{\text{cnk}}, \mathbf{e}_{2}^{\text{cnk}}\), and so on, incrementally increasing the number of chunks.

3. **Adjusting Data Mixture**: The data mixture is varied over time, starting with examples dominated by easier tasks (e.g., single chunk embedding) and gradually shifting towards those dominated by more diffic

## 7. Use AI Search MCP on GHCP

This will be needed to set up an MCP server for AI search (for vector/hybrid search), custom built in python using FastMCP, see [`./azure-ai-search-mcp`](./azure-ai-search-mcp/)

## 8. Integrate MCP with OpenWebUI

OpenWebUI doesn't support stdio MCP configurations natively, use `mcpo` python library for it to work.

OpenWebUI locally w/ uv + python: https://docs.openwebui.com/getting-started/quick-start/

Install + Run

In [ ]:
$env:DATA_DIR="C:\open-webui\data"; uvx --python 3.11 open-webui@latest serve

Updating

In [ ]:
pip install -U open-webui

Uninstall

In [ ]:
uv tool uninstall open-webui
uv cache clean

# DELETE ALL DATA
rm -rf ~/.open-webui